### MQTT Scriptlets

Short segments of setup/loop python will be delivered through MQTT and then executed 
in a system that is managing its external work using the async system, so that it can 
restart or handle incoming or outgoing MQTT messages.

This will be the ultimate in microcontroller configurability and remote operations based 
on a standard install.

The async MQTT project is at https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as

Command line code for handling MQTT
* mosquitto_sub  -v -h mqtt.local -t "scriptlet1/#"
* mosquitto_pub  -h mqtt.local -t "scriptlet1/code" -m "gothere=999"

In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [53]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/


Sent 94 lines (2691 bytes) to lib/uasyncio/queues.py.


In [4]:
# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

Sent 188 lines (7484 bytes) to aswitch.py.


In [ ]:
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

In [21]:
# Set up the wifi connection
%sendtofile config.py

from mqtt_as import config, ESP32

# Include any cross-project settings.

if ESP32:
    config['ssid'] = 'DoESLiverpool'  # EDIT if you're using ESP32
    config['wifi_pw'] = 'decafbad00'
    config['mqttchannel'] = "scriptlet1"

Sent 8 lines (228 bytes) to config.py.


In [85]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [88]:
%sendtofile scriptlet.py

@asyn.cancellable
async def setup():
    pass

@asyn.cancellable
async def loop():
    pass

Sent 7 lines (91 bytes) to scriptlet.py.


In [127]:
from mqtt_as import MQTTClient
from config import config
import uasyncio as asyncio
import asyn
import os

SERVER = "10.0.29.167"  # mqtt.local
mqttchannel = config["mqttchannel"]
client, aloop = None, None

# should we carry on with previous copy?
# We'd then have to call it
#execfile("scriptlet.py")  

# This is the MQTT batching code with headers 001/006]
slsegment = -1
slsegmentN = -1
def appendtoscriptletfile(msg):
    global slsegment, slsegmentN
    if len(msg) > 8 and msg[7] == ord("]"):
        if slsegment == -1:
            if int(msg[:3]) != 1:
                print("Bad", msg[:8])
                client.publish(mqttchannel+'/bad', msg[:8])
                return False
            slsegmentN = int(msg[4:7])
            slsegment = 1
            fin = open("scriptlet_NEW.py", "wb")
            fin.write(msg[8:])
            fin.close()
        elif int(msg[:3]) != slsegment+1 or slsegmentN != int(msg[4:7]):
            slsegment = -1
            print("Bad", msg[:8])
            client.publish(mqttchannel+'/bad', msg[:8])
            return False
        else:
            slsegment += 1
            fin = open("scriptlet_NEW.py", "ab")
            fin.write(msg[8:])
            fin.close()
            
        if slsegment != slsegmentN:
            return False
            
    else:  # Non-chunked case
        fin = open("scriptlet_NEW.py", "a")
        fin.write(msg)
        fin.close()
    slsegment = -1  # ready for start of next batch
    return True


@asyn.cancellable
async def setloop():
    try:
        await asyn.Cancellable(setup)()
        while True:
            await asyn.sleep(0.1)
            await asyn.Cancellable(loop)()
    except Exception as e:
        await client.publish(mqttchannel+'/rec', 'error '+str(e))

        
async def acallback(topic, msg):
    print((topic, msg))
    await client.publish(mqttchannel+'/rec', b'received '+msg[:8])

    if not appendtoscriptletfile(msg):
        return
    os.rename("scriptlet_NEW.py", "scriptlet.py")
    await client.publish(mqttchannel+'/rec', 'abouttocancel_all')
    await asyn.Cancellable.cancel_all()
    await client.publish(mqttchannel+'/rec', 'abouttoexec')
    try:
        execfile("scriptlet.py")
    except Exception as e:
        print(e)
        await client.publish(mqttchannel+'/rec', str(e))
        return
    
    await client.publish(mqttchannel+'/rec', 'abouttorun')
    aloop.create_task(asyn.Cancellable(setloop)())

# The callback has to be a normal function, but we can create an async task from within it using aloop
def callback(topic, msg):
    aloop.create_task(acallback(topic, msg))
    
async def conn_han(client):
    print("subscribing", (mqttchannel+'/code'))
    await client.subscribe(mqttchannel+'/code', 1)
    
async def main(client):
    await client.connect()
    n = 0
    while True:
        await asyncio.sleep(5)
        await client.publish(mqttchannel+'/beat', '{}'.format(n), qos = 1)
        n += 1

config['subs_cb'] = callback
config['connect_coro'] = conn_han
config['server'] = SERVER

MQTTClient.DEBUG = False  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
try:
    aloop.run_until_complete(main(client))
finally:
    client.close()  # Prevent LmacRxBlk:1 errors

> auth (b0)
> assoc (0)
> run (10)
.
subscribing scriptlet1/code
..(b'scriptlet1/code', b'001/006]\n@asyn.cancellable\nfrom OLED_driver import oledsho')
(b'scriptlet1/code', b'002/006]wfattext\nasync def setup():\n    print("In fsetup")')
(b'scriptlet1/code', b'003/006]\n    import time\n    oledshowfattext(["fBegin"])\n\n')
(b'scriptlet1/code', b'004/006]@asyn.cancellable\nasync def loop():\n    oledshowfa')
(b'scriptlet1/code', b'005/006]ttext(["Time", str(time.time())])\n    await asyn.s')
(b'scriptlet1/code', b'006/006]leep(2)\n')
invalid syntax
....(b'scriptlet1/code', b'001/006]\nfrom OLED_driver import oledshowfattext\n\n@asyn.ca')
(b'scriptlet1/code', b'002/006]ncellable\nasync def setup():\n    print("In fsetup"')
(b'scriptlet1/code', b'003/006])\n    import time\n    oledshowfattext(["fBegin"])\n')
.(b'scriptlet1/code', b'004/006]\n@asyn.cancellable\nasync def loop():\n    oledshowf')
(b'scriptlet1/code', b'005/006]attext(["Time", str(time.time())])\n    await asyn.')
(b'scri

Traceback (most recent call last):
  File "<stdin>", line 108, in <module>
  File "<stdin>", line 106, in <module>
  File "/lib/uasyncio/core.py", line 180, in run_until_complete
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 
